# Things to find:

## 
* scoring average
* career high
* 3-pt percentage

In [1]:
!pip install psycopg2-binary

In [2]:
# Dependencies and Setup
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as st
from scipy.stats import linregress
from sqlalchemy import create_engine


# Study data files
lebron_data_path = "../ed_project_2/Resources/lebron_playoffs.csv"

# Read the mouse data and the study results
lebron_playoff = pd.read_csv(lebron_data_path)
lebron_playoff

,game,date,series,series_game,team,opp,result,mp,fg,fga,...,orb,drb,trb,ast,stl,blk,tov,pts,game_score,plus_minus
0,1.0,4/22/06,Eastern Conference 1,1.0,CLE,WAS,W (+11),48.0,12.0,27.0,...,3.0,8.0,11.0,11.0,0.0,0.0,4.0,32.0,23.3,11.0
1,2.0,4/25/06,Eastern Conference 1,2.0,CLE,WAS,L (-5),43.0,7.0,25.0,...,2.0,7.0,9.0,2.0,2.0,3.0,10.0,26.0,6.7,-2.0
2,3.0,4/28/06,Eastern Conference 1,3.0,CLE,WAS,W (+1),48.0,16.0,28.0,...,1.0,4.0,5.0,3.0,2.0,0.0,4.0,41.0,27.4,1.0
3,4.0,4/30/06,Eastern Conference 1,4.0,CLE,WAS,L (-10),45.0,13.0,23.0,...,1.0,5.0,6.0,5.0,0.0,0.0,7.0,38.0,23.0,-16.0
4,5.0,5/3/06,Eastern Conference 1,5.0,CLE,WAS,W (+1),46.0,14.0,23.0,...,5.0,2.0,7.0,6.0,2.0,0.0,4.0,45.0,38.4,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,258.0,10/6/20,NBA Finals,4.0,LAL,MIA,W (+6),39.0,8.0,16.0,...,2.0,10.0,12.0,8.0,1.0,0.0,6.0,28.0,23.4,-2.0
258,259.0,10/9/20,NBA Finals,5.0,LAL,MIA,L (-3),42.0,15.0,21.0,...,3.0,10.0,13.0,7.0,3.0,0.0,4.0,40.0,39.1,7.0
259,260.0,10/11/20,NBA Finals,6.0,LAL,MIA,W (+13),41.0,13.0,20.0,...,3.0,11.0,14.0,10.0,1.0,0.0,1.0,28.0,29.2,18.0
260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
lebron_playoff.tail()

,game,date,series,series_game,team,opp,result,mp,fg,fga,...,orb,drb,trb,ast,stl,blk,tov,pts,game_score,plus_minus
257,258.0,10/6/20,NBA Finals,4.0,LAL,MIA,W (+6),39.0,8.0,16.0,...,2.0,10.0,12.0,8.0,1.0,0.0,6.0,28.0,23.4,-2.0
258,259.0,10/9/20,NBA Finals,5.0,LAL,MIA,L (-3),42.0,15.0,21.0,...,3.0,10.0,13.0,7.0,3.0,0.0,4.0,40.0,39.1,7.0
259,260.0,10/11/20,NBA Finals,6.0,LAL,MIA,W (+13),41.0,13.0,20.0,...,3.0,11.0,14.0,10.0,1.0,0.0,1.0,28.0,29.2,18.0
260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.0,NaN,NaN


In [4]:
lebron_playoff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   game         260 non-null    float64
 1   date         260 non-null    object 
 2   series       260 non-null    object 
 3   series_game  260 non-null    float64
 4   team         260 non-null    object 
 5   opp          260 non-null    object 
 6   result       260 non-null    object 
 7   mp           260 non-null    float64
 8   fg           260 non-null    float64
 9   fga          260 non-null    float64
 10  fgp          260 non-null    float64
 11  three        260 non-null    float64
 12  threeatt     260 non-null    float64
 13  threep       256 non-null    float64
 14  ft           260 non-null    float64
 15  fta          260 non-null    float64
 16  ftp          258 non-null    float64
 17  orb          260 non-null    float64
 18  drb          260 non-null    float64
 19  trb     

In [5]:
lebron_playoff.isna().sum()

game           2
date           2
series         2
series_game    2
team           2
opp            2
result         2
mp             2
fg             2
fga            2
fgp            2
three          2
threeatt       2
threep         6
ft             2
fta            2
ftp            4
orb            2
drb            2
trb            2
ast            2
stl            2
blk            2
tov            2
pts            1
game_score     2
plus_minus     2
dtype: int64

In [6]:
# Remove the rows with missing data, if any!
#game count
clean_lebron_data_df = lebron_playoff.dropna(how="any")
clean_lebron_data_df

,game,date,series,series_game,team,opp,result,mp,fg,fga,...,orb,drb,trb,ast,stl,blk,tov,pts,game_score,plus_minus
0,1.0,4/22/06,Eastern Conference 1,1.0,CLE,WAS,W (+11),48.0,12.0,27.0,...,3.0,8.0,11.0,11.0,0.0,0.0,4.0,32.0,23.3,11.0
1,2.0,4/25/06,Eastern Conference 1,2.0,CLE,WAS,L (-5),43.0,7.0,25.0,...,2.0,7.0,9.0,2.0,2.0,3.0,10.0,26.0,6.7,-2.0
2,3.0,4/28/06,Eastern Conference 1,3.0,CLE,WAS,W (+1),48.0,16.0,28.0,...,1.0,4.0,5.0,3.0,2.0,0.0,4.0,41.0,27.4,1.0
3,4.0,4/30/06,Eastern Conference 1,4.0,CLE,WAS,L (-10),45.0,13.0,23.0,...,1.0,5.0,6.0,5.0,0.0,0.0,7.0,38.0,23.0,-16.0
4,5.0,5/3/06,Eastern Conference 1,5.0,CLE,WAS,W (+1),46.0,14.0,23.0,...,5.0,2.0,7.0,6.0,2.0,0.0,4.0,45.0,38.4,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,256.0,10/2/20,NBA Finals,2.0,LAL,MIA,W (+10),39.0,14.0,25.0,...,2.0,7.0,9.0,9.0,1.0,1.0,0.0,33.0,30.6,7.0
256,257.0,10/4/20,NBA Finals,3.0,LAL,MIA,L (-11),39.0,9.0,16.0,...,1.0,9.0,10.0,8.0,0.0,2.0,8.0,25.0,17.8,-4.0
257,258.0,10/6/20,NBA Finals,4.0,LAL,MIA,W (+6),39.0,8.0,16.0,...,2.0,10.0,12.0,8.0,1.0,0.0,6.0,28.0,23.4,-2.0
258,259.0,10/9/20,NBA Finals,5.0,LAL,MIA,L (-3),42.0,15.0,21.0,...,3.0,10.0,13.0,7.0,3.0,0.0,4.0,40.0,39.1,7.0


In [7]:
# pull series and game_score columns using double brackets[[]]

lebron_cols_df = clean_lebron_data_df[['game','date', 'series','threep', 'pts']]

#rename columns
lebron_cols_df.columns = ['Game', 'Date', 'Series','Three-PointPercentage', 'Points']

#lebron_playoff_stats = lebron_cols_df.reindex(columns=lebron_cols_df.columns)

#view df and look at shape

lebron_playoff_stats =lebron_cols_df.set_index('Game')
lebron_playoff_stats

,Date,Series,Three-PointPercentage,Points
Game,,,,
1.0,4/22/06,Eastern Conference 1,0.250,32.0
2.0,4/25/06,Eastern Conference 1,0.167,26.0
3.0,4/28/06,Eastern Conference 1,0.600,41.0
4.0,4/30/06,Eastern Conference 1,0.583,38.0
5.0,5/3/06,Eastern Conference 1,0.000,45.0
...,...,...,...,...
256.0,10/2/20,NBA Finals,0.375,33.0
257.0,10/4/20,NBA Finals,0.200,25.0
258.0,10/6/20,NBA Finals,0.400,28.0


In [8]:
#Playoff appearances

lebron_playoffs_number_of_games = lebron_playoff_stats['Series'].count()
lebron_playoffs_number_of_games

255

In [9]:
#Average points in playoffs

lebron_playoffs = lebron_playoff_stats.groupby(['Series'])
lebron_playoffs_avg_points = lebron_playoff_stats['Points'].mean()
lebron_playoffs_avg_points

28.937254901960785

In [10]:
#Average 3-pt percentage in the playoff series

lebron_average_3pt_percentage = lebron_playoff_stats['Three-PointPercentage'].mean()
lebron_average_3pt_in_percent = lebron_average_3pt_percentage*100
lebron_average_3pt_in_percent

31.085490196078453

In [11]:
#career high

lebron_career_high = lebron_playoff_stats['Points'].max()
lebron_career_high

51.0

In [12]:
lebron_playoffs = lebron_playoff_stats.groupby(['Game'])

lebron_playoffs_number_of_games = lebron_playoff_stats['Series'].count()
lebron_playoffs_number_of_games



255

In [13]:
lebron_playoffs = lebron_playoff_stats.groupby(['Game'])
lebron_playoffs_avg_points = lebron_playoff_stats['Points'].mean()
lebron_playoffs_avg_points

28.937254901960785

In [14]:
lebron_playoffs = lebron_playoff_stats.groupby(['Game'])
lebron_average_3pt_percentage = lebron_playoff_stats['Three-PointPercentage'].mean()
lebron_average_3pt_in_percent = lebron_average_3pt_percentage*100
lebron_average_3pt_in_percent


31.085490196078453

In [15]:
lebron_playoffs = lebron_playoff_stats.groupby(['Game'])
lebron_playoffs_career_high = lebron_playoff_stats['Points'].max()
lebron_playoffs_career_high

51.0

In [16]:
lebron_stats_df = lebron_playoff_stats
lebron_stats_df

,Date,Series,Three-PointPercentage,Points
Game,,,,
1.0,4/22/06,Eastern Conference 1,0.250,32.0
2.0,4/25/06,Eastern Conference 1,0.167,26.0
3.0,4/28/06,Eastern Conference 1,0.600,41.0
4.0,4/30/06,Eastern Conference 1,0.583,38.0
5.0,5/3/06,Eastern Conference 1,0.000,45.0
...,...,...,...,...
256.0,10/2/20,NBA Finals,0.375,33.0
257.0,10/4/20,NBA Finals,0.200,25.0
258.0,10/6/20,NBA Finals,0.400,28.0


In [17]:
lebron_playoff_stats.to_csv('lebron_stats.csv')

# Load Data in PosgresSQL

In [18]:
# Create a connection to the database so we can upload our data into the proper tables located in our PostgreSQL server
# The name of our database is 'lebron_vs-steph' and we must define the path the database using our local pw and login settings
# assing this string to the variable 'connection_string'
connection_string = "postgres:postgres@localhost:5432/lebron_vs_steph"

# create a connection to the server using create_engine() method and name it 'engine'
engine = create_engine(f'postgresql://{connection_string}')


In [19]:
engine.table_names()

<ipython-input-19-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['lebron_stats', 'steph_stats']

# Postgres Query

In [22]:
pd.read_sql_query('select * from lebron_stats', con=engine).head()

,game,date,series,threepointpercentage,points
0,1,2006-04-22,Eastern Conference 1,0.250,32
1,2,2006-04-25,Eastern Conference 1,0.167,26
2,3,2006-04-28,Eastern Conference 1,0.600,41
3,4,2006-04-30,Eastern Conference 1,0.583,38
4,5,2006-05-03,Eastern Conference 1,0.000,45


In [23]:
pd.read_sql_query('select * from steph_stats', con=engine).head()

,season_year,date,threepointers,pts
0,2012-2013,Thu 5/16,25.0,22
1,2012-2013,Tue 5/14,14.3,9
2,2012-2013,Sun 5/12,50.0,22
3,2012-2013,Fri 5/10,33.3,16
4,2012-2013,Wed 5/8,33.3,22


In [26]:
#Steph's all time career playoff three point percentage 
pd.read_sql_query('select AVG(threepointers) from steph_stats', con=engine)

,avg
0,40.050926


In [27]:
#Lebron's all time career playoff three point percentage
pd.read_sql_query('select AVG(threepointpercentage) from lebron_stats', con=engine)

,avg
0,0.310855


In [28]:
#Steph Curry's playoff career high
pd.read_sql_query('select MAX(pts) from steph_stats', con=engine)

,max
0,47


In [30]:
#Lebron James playoff career high
pd.read_sql_query('select MAX(points) from lebron_stats', con=engine)

,max
0,51


In [31]:
#Steph's playoff scoring average
pd.read_sql_query('select AVG(pts) from steph_stats', con=engine)

,avg
0,26.574074


In [32]:
#Lebron's playoff scoring average
pd.read_sql_query('select AVG(points) from lebron_stats', con=engine)

,avg
0,28.937255
